In [44]:
#Import the dependencies
import pandas as pd
import gmaps
import requests
from config import g_key

In [45]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,New Norfolk,AU,2020-09-28 04:03:10,-42.78,147.06,59.00,58,20,16.11
1,1,Marawi,PH,2020-09-28 04:07:33,8.00,124.29,84.99,76,97,4.00
2,2,Broken Hill,AU,2020-09-28 04:07:34,-31.95,141.43,64.40,34,43,10.29
3,3,East London,ZA,2020-09-28 04:07:34,-33.02,27.91,48.20,81,1,6.93
4,4,Butaritari,KI,2020-09-28 04:07:35,3.07,172.79,81.70,79,1,15.46
5,5,Castro,BR,2020-09-28 04:07:36,-24.79,-50.01,63.86,81,100,5.10
6,6,Indianola,US,2020-09-28 04:07:36,41.36,-93.56,53.01,92,89,8.48
7,7,Jamestown,US,2020-09-28 04:05:08,42.10,-79.24,64.40,77,1,8.05
8,8,Barrow,US,2020-09-28 04:07:37,71.29,-156.79,32.00,96,90,10.29
9,9,Dwarka,IN,2020-09-28 04:05:29,22.24,68.97,83.08,61,18,6.71


In [46]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [47]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [48]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[['Lat', 'Lng']]
#Get the maximum temperature
max_temp = city_data_df['Max Temp']
# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating =False, max_intensity=300, point_radius=2.5)
# Add the heatmap layer
fig.add_layer(heat_layer)
# Call the figure to plot the data 
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
# Heatmap of humidity
# Get the humidity

humidity = city_data_df['Humidity']
# Assign the figure variable
figs = gmaps.figure(center =(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights =humidity,dissipating =False, max_intensity =300, point_radius =2.5)
# Add the heatmap layer
fig.add_layer(heat_layer)
#Call the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# Heatmap of cloudiness
# Get the cloudiness
clouds = city_data_df['Cloudiness']
# Assign the figure variable
figs = gmaps.figure(center =(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights =clouds,dissipating =False, max_intensity =300, point_radius =1)
# Add the heatmap layer
fig.add_layer(heat_layer)
#Call the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# Heatmap of Wind speed
# Get the wind speed
wind = city_data_df['Wind Speed']
# Assign the figure variable
figs = gmaps.figure(center =(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights =wind,dissipating =False, max_intensity =300, point_radius =1)
# Add the heatmap layer
fig.add_layer(heat_layer)
#Call the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [52]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?60
What is the maximum temperature you would like for your trip?65


In [53]:
preffered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] > min_temp)& (city_data_df['Max Temp']<max_temp)]

In [54]:
preffered_cities_df = preffered_cities_df.dropna()
preffered_cities_df.shape

(49, 10)

In [55]:
hotel_df = preffered_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Broken Hill,AU,64.40,-31.95,141.43,
5,Castro,BR,63.86,-24.79,-50.01,
7,Jamestown,US,64.40,42.10,-79.24,
12,Karacakoy,TR,64.00,41.40,28.38,
13,Colares,PT,62.01,38.80,-9.45,


In [56]:
# Set parameters to search for a hotel.
params = {
    'radius':5000,
    'type':'lodging',
    'key':g_key,
    'location':''
    
}

In [57]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row['Lat']
    lng = row['Lng']
    # Add the latitude and longitude to location key for the params dictionary.
    params['location'] = f'{lat},{lng}'
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel name not found...skipping.')

Hotel name not found...skipping.
Hotel name not found...skipping.
Hotel name not found...skipping.
Hotel name not found...skipping.


In [ ]:
info_box_template ="""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [68]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))